### Logistic Reg Model - Binary Classifier


In [13]:
# IDEAS: 
# https://stackoverflow.com/questions/38077190/how-to-increase-the-model-accuracy-of-logistic-regression-in-scikit-python

# scale the columns
# https://www.keboola.com/blog/logistic-regression-machine-learning
# https://www.kdnuggets.com/2020/04/data-transformation-standardization-normalization.html
# drop features/remove multicoliniarity features

# look to use class weight/class imbalances

# try SVM; maybe we cannot linearly seperate yes vs no leads column?

# add features???

In [14]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

# Import and read the csv.
#df = pd.read_csv('model_df1_official.csv', index_col=[0])
# OR: 
df = pd.read_csv('nnML_tiers.csv', index_col=[0])
df.head()

,state,brand,spend,platform,date_start,Impressions,leads,link_clicks,reach,agency,State Tiers,Agency Tiers
770,CA,B1,7.28,Facebook Ads,5/11/2021,427,0.0,2,403.0,Agency 4,4,4
771,CA,B1,13.32,Facebook Ads,6/23/2021,1042,1.0,7,962.0,Agency 4,4,4
772,CA,B1,10.11,Facebook Ads,6/13/2021,337,0.0,0,321.0,Agency 4,4,4
773,CA,B1,0.00,Facebook Ads,4/20/2021,0,1.0,0,0.0,Agency 4,4,4
774,CA,B1,7.35,Facebook Ads,5/11/2021,379,1.0,0,364.0,Agency 4,4,4


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157672 entries, 770 to 160484
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   state         157672 non-null  object 
 1   brand         157672 non-null  object 
 2   spend         157672 non-null  float64
 3   platform      157672 non-null  object 
 4   date_start    157672 non-null  object 
 5   Impressions   157672 non-null  int64  
 6   leads         157672 non-null  float64
 7   link_clicks   157672 non-null  int64  
 8   reach         157672 non-null  float64
 9   agency        157672 non-null  object 
 10  State Tiers   157672 non-null  int64  
 11  Agency Tiers  157672 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 15.6+ MB


In [135]:
df2 = df.drop(columns=['state', 'agency', 'brand', 'platform', 'date_start', 'Impressions'])
df2

,leads,link_clicks,reach,State Tiers,Agency Tiers
770,0.0,2,403.0,4,4
771,1.0,7,962.0,4,4
772,0.0,0,321.0,4,4
773,1.0,0,0.0,4,4
774,1.0,0,364.0,4,4
...,...,...,...,...,...
160480,0.0,2,30.0,1,1
160481,0.0,0,24.0,1,1
160482,0.0,4,567.0,1,1
160483,0.0,2,691.0,1,1


In [136]:
# steps: 
# create the model with LogisticRegression()
# train the model with model.fit()
# make predictions with model.predict()
# validate with accuracy_score()

### Seperate features, x and target, y

In [137]:
y = df2["leads"]
X = df2.drop(columns="leads")

### Split into train and test features

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(118254, 4)

### Create Log Reg Model 

In [139]:
# edit the params as needed
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='liblinear',
                                max_iter=200,
                                random_state=1,
                                class_weight='balanced',
                               multi_class='auto', 
                               penalty='l1')

### Fit/train or model using training data

In [140]:
classifier.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=200, penalty='l1',
                   random_state=1, solver='liblinear')

### Make predictions

In [141]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,1.0,1.0
1,1.0,1.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
5,1.0,1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,0.0


### Validate

In [142]:
# accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5484042823075752


In [109]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)
# TP = 0
# FP = 0
# FN = 17,460
# TN = 21,958

# precision = TP/(TP+FP), in this case a precision of 0 means what?
# sensitivity = TP/(TP+FN), in this case a score of 0 means what? = recall

[[ 2598 14862]
 [ 2931 19027]]


In [110]:
# classification report
report = classification_report(y_test, y_pred)
print(report)

### note: 
# ytest are the outcomes, either yes or no on leads column target 
# y_pred are the predictions
# do we read the accuracy score as 56% on predictions as a lead? 
# F1 = harmonic mean, takes sensitivity and precision = .72 here means what?

              precision    recall  f1-score   support

         0.0       0.47      0.15      0.23     17460
         1.0       0.56      0.87      0.68     21958

    accuracy                           0.55     39418
   macro avg       0.52      0.51      0.45     39418
weighted avg       0.52      0.55      0.48     39418

